# Data Extraction - Comprehensive Azure AI Document Intelligence + Azure OpenAI GPT-4o with Vision

This sample demonstrates how to build a comprehensive process to extract structured data from any document using Azure AI Document Intelligence and Azure OpenAI's GPT-4o model with vision capabilities.

![Data Extraction](../../../images/extraction-comprehensive.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes the specific extraction instruction for the type of document, the text content, and each document page as a base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from the document page images using Azure OpenAI's GPT-4o model.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **pdf2image** for converting a PDF file into a set of images per page.
- **azure-ai-documentintelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.comparison** to compare the output of the extraction process with expected results.
- **modules.document_intelligence_confidence** to evaluate the confidence of the extraction process based on the extracted structured output and the analysis result from Azure AI Document Intelligence.
- **modules.document_processing_result** to store the results of the extraction process as a file.
- **modules.openai_confidence** to calculate the confidence of the classification process based on the `logprobs` response from the API request.
- **modules.bankdocs** to provide the expected structured output JSON schema for bank account documents.
- **modules.utils** `Stopwatch` to measure the end-to-end execution time for the classification process.

In [15]:
import sys
sys.path.append('../../') # Import local modules

from IPython.display import display, Markdown
import os
import pandas as pd
from dotenv import dotenv_values
from pdf2image import convert_from_bytes
import base64
import io
from openai import AzureOpenAI
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import json

from modules.app_settings import AppSettings
from modules.utils import Stopwatch
from modules.accuracy_evaluator import AccuracyEvaluator
from modules.comparison import get_extraction_comparison
from modules.confidence import merge_confidence_values
from modules.document_intelligence_confidence import evaluate_confidence as di_evaluate_confidence
from modules.openai_confidence import evaluate_confidence as oai_evaluate_confidence
from modules.vehicle_insurance_policy import VehicleInsurancePolicy
from modules.document_processing_result import DataExtractionResult

from modules.bankdocs import FinancialReport

from modules.taxdoc import TaxDocument

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [16]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-10-01-preview" # Requires the latest API version for structured outputs.
)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=settings.ai_services_endpoint,
    credential=credential
)

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on each page of a [Bank Document](../../assets/vehicle_insurance/policy_1.pdf).

> **Note**: More insurance policy examples can be found in the [assets folder](../../assets/vehicle_insurance). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [17]:
path = f"{working_dir}/samples/assets/taxforms/"
pdf_files = [f for f in os.listdir(path) if f.endswith('.pdf')][0]
# metadata_fname = "bankdoc.json" # Change this to the file you want to evaluate
# metadata_fpath = f"{path}{metadata_fname}"

# with open(metadata_fpath, 'r') as f:
#     data = json.load(f)
    
# expected = FinancialReport(**data['expected'])
# pdf_fname = data['fname']
# pdf_fpath = f"{path}{pdf_fname}"

# tax_evaluator = AccuracyEvaluator(match_keys=[])

## Extract data from the document

The following code block executes the data extraction process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Use pdf2image to convert the document's pages into images per page as base64 strings.
4. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the images.

In [18]:
with Stopwatch() as di_stopwatch:
    pdf_file_path = os.path.join(path, pdf_files)
    with open(pdf_file_path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout",
            analyze_request=f,
            output_content_format=ContentFormat.MARKDOWN,
            content_type="application/pdf"
        )
    
    result: AnalyzeResult = poller.result()

markdown = result.content

# Write the markdown output to a file
markdown_filename = f"{pdf_files[0].split('.')[0]}_md"
with open(f"{working_dir}/samples/assets/taxforms/{markdown_filename}.md", "w") as md_file:
    md_file.write(markdown)

In [19]:
# Prepare the user content for the OpenAI API including any specific details for processing this type of document, text, and the document page images.
user_content = []
user_content.append({
    "type": "text",
    "text": f"""Extract the data from this tax document. 
    - If a value is not present, provide null.
    - Dates should be in the format YYYY-MM-DD."""
})

user_content.append({
    "type": "text",
    "text": markdown
})

In [20]:
with Stopwatch() as image_stopwatch:
    for pdf_file in pdf_files:
        pdf_file_path = os.path.join(path, pdf_files)
        document_bytes = open(pdf_file_path, "rb").read()
        page_images = convert_from_bytes(document_bytes)
        for page_image in page_images:
            byteIO = io.BytesIO()
            page_image.save(byteIO, format='PNG')
            base64_data = base64.b64encode(byteIO.getvalue()).decode('utf-8')

            user_content.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_data}"
                }
            })
            page_image.save(f"{working_dir}/samples/assets/taxforms/page_{page_images.index(page_image) + 1}.png")

In [21]:
with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that extracts data from documents.",
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        response_format=FinancialReport,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1,
        logprobs=True # Enabled to determine the confidence of the response.
    )

In [24]:
# Write the model output to a JSON file
output_filename = f"{pdf_files[0].split('.')[0]}_output.json"
output_filepath = os.path.join(path, output_filename)

with open(output_filepath, "w") as json_file:
    json.dump(completion.choices[0].message.content, json_file, indent=4)

### Understanding the Structured Outputs JSON schema

Using [Pydantic's JSON schema feature](https://docs.pydantic.dev/latest/concepts/json_schema/), the [Insurance Policy](../../modules/vehicle_insurance_policy.py) data model is automatically converted to a JSON schema when applied to the `response_format` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach using Pydantic makes it easier for developers to manage the data structure in code, with helpful descriptions and examples that will be included in the final JSON schema.

Demonstrated below, you can see how the Insurance Policy data model is understood by the OpenAI request:

In [22]:
# Highlight the schema sent to the OpenAI model
print(json.dumps(FinancialReport.model_json_schema(), indent=2))

{
  "$defs": {
    "AccountDetails": {
      "properties": {
        "account_name": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Name of the account",
          "title": "Account Name"
        },
        "account_number": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Account number",
          "title": "Account Number"
        }
      },
      "required": [
        "account_name",
        "account_number"
      ],
      "title": "AccountDetails",
      "type": "object"
    },
    "AccountGrowth": {
      "properties": {
        "value_at_year_end_2022": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "descript

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction based on combining the confidence scores of the Azure AI Document Intelligence layout analysis and the log probability of the output generated by Azure OpenAI's GPT-4o model.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [23]:
# Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(Markdown(markdown))

a

Test Financial Services Inc.
Private Wealth Management 999
Test Location
Apt. 900 - Test BUILDING
Maples IN yyyy00000
CHD0000000000 999999 9

Test Test Program
June 2023

Test BANK & Test N.A.
Test Street ABC Eve.
3111 BLVD SK CA 00000-1989

Account name: Test BANK & TRUST N.A.

Test Name

Account number: 9K 98981212

Your Financial Advisor:

THE Test GROUP

Phone: 91919191/12121212

Questions about your statement? Call
your Financial Advisor or the RMA
ResourceLine at 800-RMA-9999,
account 44XXXX999.

Visit our website:
www.Test.com/financialservices

Items for your attention

4 Help protect yourself from fraud and
review bank, credit card and brokerage
statements regularly. Also, get your free
credit report annually from
www.annualcreditreport.com.

Value of your account
on May 31 ($)

on June 30 ($)


<table>
<tr>
<th>Your assets</th>
<th>2,954,398.87</th>
<th>3,080,882.85</th>
</tr>
<tr>
<td>Your liabilities</td>
<td>0.00</td>
<td>0.00</td>
</tr>
<tr>
<td>Value of your account</td>
<td>$2,954,398.87</td>
<td>$3,080,882.85</td>
</tr>
<tr>
<td>Accrued interest in value above</td>
<td>$8,806.88</td>
<td>$6,809.65</td>
</tr>
</table>


As a service to you, your portfolio value of
$3,080,882.85 includes accrued interest.


<table>
<tr>
<th>Dec 2021</th>
<th>Dec 2022</th>
</tr>
<tr>
<td></td>
<td>May 2023</td>
</tr>
<tr>
<td></td>
<td>Jun 2023</td>
</tr>
</table>


# Tracking the value of your account

$ Thousands

3,577.3

2,852.8

2,954.4

3,080.9

Member SIPC

CNP4000000001010104050803070 666666666 00000004 99999999065 2T02798NA0 111100


<figure>

PDF Merger
2

<!-- PageNumber="Page 1 of 54" -->

</figure>


<!-- PageBreak -->


# Sources of your account growth during 2023


<table>
<tr>
<td>Value of your account</td>
<td></td>
</tr>
<tr>
<td>at year end 2022</td>
<td>$2,852,834.99</td>
</tr>
<tr>
<td>Net deposits and</td>
<td></td>
</tr>
<tr>
<td>withdrawals</td>
<td>-$28,316.30</td>
</tr>
<tr>
<td>Your investment return:</td>
<td></td>
</tr>
<tr>
<td>Dividend and</td>
<td></td>
</tr>
<tr>
<td>interest income</td>
<td>$30,053.15</td>
</tr>
<tr>
<td rowspan="2">Change in value of accrued interest</td>
<td></td>
</tr>
<tr>
<td>-$684.16</td>
</tr>
<tr>
<td rowspan="2">Change in market value</td>
<td></td>
</tr>
<tr>
<td>$226,995.17</td>
</tr>
</table>


Value of your account

<!-- PageFooter="CNP700030001015555550803070 0N0P070000000403050100070 99990004 0623" -->


<figure>

PDF Merger

<!-- PageNumber="Page 2 of 54" -->

</figure>


<!-- PageBreak -->


# Your account balance sheet

The value of your account includes assets held at ABC and certain assets held away from ABC. See page 1 for
more information.


## Summary of your assets


<table>
<tr>
<th></th>
<th>Value on June 30 ($)</th>
<th>Percentage of your account</th>
</tr>
<tr>
<td>A Cash and money balances</td>
<td>96,118.72</td>
<td>3.12%</td>
</tr>
<tr>
<td>B Cash alternatives</td>
<td>0.00</td>
<td>0.00%</td>
</tr>
<tr>
<td>C Equities</td>
<td>2,442,159.48</td>
<td>79.27%</td>
</tr>
<tr>
<td>D Fixed income</td>
<td>542,604.65</td>
<td>17.61%</td>
</tr>
<tr>
<td>E Non-traditional</td>
<td>0.00</td>
<td>0.00%</td>
</tr>
<tr>
<td>F Commodities</td>
<td>0.00</td>
<td>0.00%</td>
</tr>
<tr>
<td>G Other</td>
<td>0.00</td>
<td>0.00%</td>
</tr>
<tr>
<td>Total assets</td>
<td>$3,080,882.85</td>
<td>100.00%</td>
</tr>
</table>


Value of your account

$3,080,882.85


<figure>
<figcaption>Your current asset Test</figcaption>

A

D

C

</figure>


4 Cash and money balances may include
available cash balances, ABC Bank USA deposit
account, ABC FDIC Insured Deposit Program Bank
accounts, ABC Insured Sweep Program Bank
accounts, ABC AG EFG Branch deposit account
balances and money market mutual fund sweep
balances. See the Important information about your
statement on the last two pages of this statement
for details about those balances.


## Eye on the markets


<table>
<tr>
<th rowspan="2">Index</th>
<th colspan="2">Percentage change</th>
</tr>
<tr>
<th>June 2023</th>
<th>Year to date</th>
</tr>
<tr>
<td>S&amp;P 500</td>
<td>6.61%</td>
<td>16.89%</td>
</tr>
<tr>
<td>Russell 3000</td>
<td>6.83%</td>
<td>16.17%</td>
</tr>
<tr>
<td>MSCI - Europe, Australia &amp; Far East</td>
<td>4.58%</td>
<td>12.13%</td>
</tr>
<tr>
<td>Barclays Capital U.S. Aggregate Bond Index</td>
<td>-0.36%</td>
<td>2.09%</td>
</tr>
</table>


Interest rates on June 30, 2023

3-month Treasury bills: 5.13%
One-month LIBOR: 5.22%

<!-- PageFooter="Member SIPC" -->
<!-- PageFooter="CNP4000000001010104050803070 666666666 00000004 99999999065 2T02798NA0 111100" -->


<figure>

PDF Merger

2

<!-- PageNumber="Page 1 of 54" -->

</figure>

In [ ]:
# # Gets the parsed VehicleInsurancePolicy object from the completion response.
# tax_doc = completion.choices[0].message.parsed

# expected_dict = expected.to_dict()
# tax_doc_dict = tax_doc.to_dict()

In [ ]:
# # Determines the accuracy of the extracted data against the expected values.
# accuracy = tax_evaluator.evaluate(expected=expected_dict, actual=tax_doc_dict)

In [ ]:
# # Determines the confidence of the extracted data using both the OpenAI and Azure Document Intelligence responses.
# di_confidence = di_evaluate_confidence(tax_doc_dict, result)
# oai_confidence = oai_evaluate_confidence(tax_doc_dict, completion.choices[0])

# confidence = merge_confidence_values(di_confidence, oai_confidence)

In [ ]:
# # Gets the total execution time of the data extraction process.
# total_elapsed = di_stopwatch.elapsed + image_stopwatch.elapsed + oai_stopwatch.elapsed

# # Gets the prompt tokens and completion tokens from the completion response.
# prompt_tokens = completion.usage.prompt_tokens
# completion_tokens = completion.usage.completion_tokens

In [ ]:
# # Save the output of the data extraction result.
# extraction_result = DataExtractionResult(tax_doc_dict, confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

# with open(f"{working_dir}/samples/assets/taxforms/comprehensive.{pdf_fname}.json", "w") as f:
#     f.write(extraction_result.to_json(indent=4))

In [ ]:
# # Display the outputs of the data extraction process.
# df = pd.DataFrame([
#     {
#         "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
#         "Confidence": f"{confidence['_overall'] * 100:.2f}%",
#         "Execution Time": f"{total_elapsed:.2f} seconds",
#         "Document Intelligence Execution Time": f"{di_stopwatch.elapsed:.2f} seconds",
#         "Image Pre-processing Execution Time": f"{image_stopwatch.elapsed:.2f} seconds",
#         "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
#         "Prompt Tokens": prompt_tokens,
#         "Completion Tokens": completion_tokens
#     }
# ])

# display(df)
# display(get_extraction_comparison(expected_dict, tax_doc_dict, confidence, accuracy['accuracy']))

,Accuracy,Confidence,Execution Time,Document Intelligence Execution Time,Image Pre-processing Execution Time,OpenAI Execution Time,Prompt Tokens,Completion Tokens
0,37.50%,98.24%,23.10 seconds,5.76 seconds,0.94 seconds,16.40 seconds,4187,123


,Field,Expected,Extracted,Confidence,Accuracy
0,address_city,Toronto,Waterloo,99.50%,Mismatch
1,address_postal_code,A1B 2C3,N2T2M5,96.31%,Mismatch
2,address_province,Ontario,ON,99.40%,Mismatch
3,address_street,Main St,123 Fake Street,99.20%,Mismatch
4,business_province,,Ontario,99.50%,Mismatch
5,current_province,,Ontario,99.50%,Mismatch
6,date_of_birth,1980-01-01,1977-10-05,100.00%,Mismatch
7,first_name,John,John,98.40%,Match
8,language_of_correspondence,English,English,99.60%,Match
9,last_name,Doe,Doe,99.40%,Match
